In [ ]:
import os

# get a list of all files in the directory
directory = "Humor,Hist,Media,Food/"
files = os.listdir(directory)

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
def preprocess(text):
    tokenizer = nltk.RegexpTokenizer(r"\w+")    # remove punctuations
    tokens = tokenizer.tokenize(text)

    tokens = [token.lower() for token in tokens]    # lower case
    # tokens_no_sw = [word for word in tokens if not word in stopwords.words()]    # remove stop words
    # return tokens_no_sw
    return tokens

In [ ]:
token_doc_id = {}
token_freq = {}

for file_name in files:    # for each file
    file_text = open(directory+file_name,'r',encoding='ISO-8859-1').read()    # read the file
    tokens = preprocess(file_text)
    # print(tokens)
    for token in tokens:
        if token not in token_doc_id:
            token_doc_id[token] = {}
            token_freq[token] = 0
        if file_name not in token_doc_id[token]:
            token_doc_id[token][file_name] = 0
        token_doc_id[token][file_name] += 1
        token_freq[token]+=1

In [ ]:
def and_op(list_left,list_right):
    # get intersection of left and right
    intersection = []
    i = 0
    j = 0
    while i<len(list_left) and j<len(list_right):
        if list_left[i]==list_right[j]:
            intersection.append(list_left[i])
            i+=1
            j+=1
        elif list_left[i]<list_right[j]:
            i+=1
        else:
            j+=1
    return intersection

In [ ]:
def or_op(list_left,list_right):
    # get union of left and right
    union = []
    i = 0
    j = 0
    while i<len(list_left) and j<len(list_right):
        if list_left[i]==list_right[j]:
            union.append(list_left[i])
            i+=1
            j+=1
        elif list_left[i]<list_right[j]:
            union.append(list_left[i])
            i+=1
        else:
            union.append(list_right[j])
            j+=1
    return union

In [ ]:
def not_op(word):
    # get complement of given list
    complement = []
    for f in files:
        if f not in token_doc_id[word]:
            complement.append(file)
    return complement

In [ ]:
def do_operation(query,opers):
    # do query and operations
    num_ops = len(opers)
    query = query[:num_ops+1]

    query_doc_id = [0]*len(query)
    query_docid_after_nots = [token_doc_id[query[0]].keys()]
    for i in range(len(opers)):
        if opers[i]=='AND NOT' or opers[i]=='OR NOT':
            query_docid_after_nots.append(not_op(token_doc_id[query[i+1]].keys()))
        else:
            query_docid_after_nots.append(token_doc_id[query[i+1]].keys())
    
    obj1 = in_to_postfix()
    postfixexp = obj1.postfix(query_docid_after_nots,opers)

    obj2 = evaluate()
    final_list_docs = obj2.evaluate_postfix(postfixexp)

    return final_list_docs

In [ ]:
def preprocess_operations(ops):
    return [op.upper() for op in ops]

def preprocess_query(text):
    tokenizer = nltk.RegexpTokenizer(r"\w+")    # remove punctuations
    tokens = tokenizer.tokenize(text)
    tokens = [token.lower() for token in tokens]    # lower case
    tokens_no_sw = [word for word in tokens if not word in stopwords.words()]    # remove stop words
    return tokens_no_sw

# query
with open('Q1_input.txt','r') as inp:
    number_of_queries = int(inp.readline())
    for i in range(number_of_queries):
        input_text = inp.readline().strip().split(" ")
        input_ops = inp.readline().split(", ")
        ops_pp = preprocess_operations(input_ops)
        print(do_operation(input_text,ops_pp))

    # t = 'lion stood thoughtfully for a moment'
    # print(preprocess(t))

OR
OR NOT


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=c98dd744-d9d5-4215-badf-4f45241c19af' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>